<b>Exercise 1: Comparison of Python and NEURON syntax.</b>
<br><br>
We continue with the same Hodgkin-Huxley model as introduced in Lecture 4.
$$C_{\rm m}\frac{{\rm d}V}{{\rm d}t} = g_{\rm Na} m_{\infty}(V)h(E_{\rm Na}-V) + g_{\rm K} (E_{\rm K}-V)$$
where $h$ is controlled by:
$$ \frac{{\rm d}h}{{\rm d}t} = \alpha_h (1-h)-\beta_h h$$


In [1]:
########################################################
############## L4, Exercise 5 reprise  #################
########### Python solver implementation ###############
########################################################

import matplotlib.pyplot as plt
plt.ion()
from scipy.integrate import odeint
from pylab import *
import time

#Definition of constant parameters and initial values
C_m = 0.05 # nF    
E_Na = 50;
E_K = -80;
g_Na = 0.8; # uS, 20pS/channel*40,000 channels pr cell. 
g_K = 0.1
I_amp = 0;
h0 = 0.5;
V0 = -50;

dm = 15
dh = -5
dam = dm
dah = dh;
dbm = -dm
dbh = -dh;

#Definition of functions needed for integration of the differential equation
def m_rates(V):
    alpha = exp((V-Vam)/dam)
    beta  = exp((V-Vbm)/dbm)
    return alpha, beta
def h_rates(V):
    alpha = exp((V-Vah)/dah)
    beta  = exp((V-Vbh)/dbh)
    return alpha, beta
def I_Na(V, h):
    alpha_m, beta_m = m_rates(V)
    return g_Na*alpha_m/(alpha_m+beta_m)*h*(V - E_Na)
def I_K(V):
    return g_K*(V - E_K)

def dY_dt(Y, t=0):
    V, h = Y
    d_V = (-I_Na(V, h)-I_K(V))/C_m
    alpha_h, beta_h = h_rates(V)
    d_h = alpha_h*(1-h) - h*beta_h;
    return [d_V, d_h]

def solve(dt, tstop, V0_l, h0_l, Vam_l, Vbm_l, Vah_l, Vbh_l):
    global  V0, h0, Vam, Vbm, Vah, Vbh
    timenow = time.time()
    V0 = V0_l
    h0 = h0_l
    Vam = Vam_l
    Vbm = Vbm_l
    Vah = Vah_l
    Vbh = Vbh_l
    Y = odeint(dY_dt, [V0,h0], [i*dt for i in range(0,int(tstop/dt))])

    V = Y[:, 0]
    h = Y[:, 1]
    print('Python solver run in '+str(time.time()-timenow)+' seconds')
    
    return V, h


In [2]:
%%html
<style type='text/css'>
.widget-inline-hbox .widget-label {
      max-width: 250px;
      min-width: 250px;
}
</style>

In [3]:
#This command compiles all .mod mechanism files in the working directory, one of which is L4model.mod that we will use
!nrnivmodl

/home/user/SUURPh-summer-school/L17C-18C
K_Pst.mod L4model.mod myhh.mod Nap_Et2.mod
K_Pst.mod L4model.mod myhh.mod Nap_Et2.mod
"/opt/conda/share/nrn/libtool"  --mode=compile gcc -DHAVE_CONFIG_H  -I. -I.. -I"/opt/conda/include/nrn" -I"/opt/conda/lib"      -m64 -fPIC -c -o mod_func.lo mod_func.c
libtool: compile:  gcc -DHAVE_CONFIG_H -I. -I.. -I/opt/conda/include/nrn -I/opt/conda/lib -m64 -fPIC -c mod_func.c  -fPIC -DPIC -o .libs/mod_func.o
"/opt/conda/share/nrn/libtool"  --mode=link gcc -module  -m64 -fPIC    -Wl,-rpath,/opt/conda/lib -L/opt/conda/lib -o libnrnmech.la -rpath "/opt/conda/lib"  K_Pst.lo L4model.lo myhh.lo Nap_Et2.lo mod_func.lo  -L"/opt/conda/lib" -lnrnoc -loc -lmemacs -lnrnmpi -lscopmath -lsparse13 -lreadline -lcurses -L"/opt/conda/lib" "/opt/conda/lib/libnrniv.la" -livoc -lneuron_gnu -lmeschach -lsundials -livos      -lm -ldl
libtool: link: rm -fr  .libs/libnrnmech.la .libs/libnrnmech.lai .libs/libnrnmech.so .libs/libnrnmech.so.0 .libs/libnrnmech.so.0.0.0
libtool: link:

In [4]:
########################################################
################# NEURON implementation ################
##### See L4model.mod for the membrane mechanisms ######
########################################################

#Import the NEURON library
from neuron import h
from ipywidgets import widgets, fixed, Layout

#Instead of putting the hoc code into a "myneuron.hoc" as in the previous exercise, we can run all hoc code
#in triple quotations within the hoc interpreter neuron.h()
def solve_NEURON(dt,tstop,Vam,Vbm,Vah,Vbh):
    timenow = time.time()
    h("""load_file("stdlib.hoc")
load_file("stdrun.hoc")

objref cvode
cvode = new CVode()
cvode.active(0)

create soma
access soma

soma insert L4model
soma {Ra = 1 cm = 0.5 diam = 10 L = 31.8309886 gnabar_L4model = 0.008 gkbar_L4model = 0.001}
soma nseg = 1
ena = 50
ek = -80
soma Vam_L4model = """+str(Vam)+"""
soma Vbm_L4model = """+str(Vbm)+"""
soma Vah_L4model = """+str(Vah)+"""
soma Vbh_L4model = """+str(Vbh)+"""

dt = """+str(dt)+"""
tstop = 50

objref Vrec, hrec

Vrec = new Vector()
Vrec.record(&v(0.5),dt)
hrec = new Vector()
hrec.record(&h_L4model(0.5),dt)
v_init = -65

init()
run()
""")
    print('NEURON run in '+str(time.time()-timenow)+' seconds')
    return array(h.Vrec), array(h.hrec)

In [5]:
#######################################################
# Definition of a function that runs both simulations #
#######################################################
def runall(Vam=-72,Vbm=-10,Vah=-80,Vbh=-20):

    dt = 0.0001
    Vs, hs = solve(dt, 50, -65, 0.5, Vam, Vbm, Vah, Vbh)
    Vs_N, hs_N = solve_NEURON(dt, 50, Vam, Vbm, Vah, Vbh)
    times = [dt*i for i in range(0,len(Vs))]
    Vs_N = Vs_N[0:-1] #Remove the last value to
    hs_N = hs_N[0:-1] #make same length as the Python solution
        
    # Plots
    plt.subplot(2,2,1)
    plt.plot(times, Vs)
    plt.ylabel('Python solver')
    plt.title('$V_m$ (mV)')
    plt.xticks([0,20,40],[])
    plt.xlim([0, 50])
    plt.ylim([-80, 40])

    plt.subplot(2,2,2)
    plt.plot(times, hs)
    plt.title('$h$')
    plt.xticks([0,20,40],[])
    plt.xlim([0, 50])
    plt.ylim(0,1)

    plt.subplot(2,2,3)
    plt.plot(times,Vs_N)
    plt.xlabel('$t$ (ms)')
    plt.ylabel('NEURON')
    plt.xticks([0,20,40])
    plt.xlim([0, 50])
    plt.ylim([-80, 40])

    plt.subplot(2,2,4)
    plt.plot(times,hs_N)
    plt.xlabel('$t$ (ms)')
    plt.xticks([0,20,40])
    plt.xlim([0, 50])
    plt.ylim(0,1)


In [6]:
# Make a slider to run function runall() at each release of mouse button
slider = widgets.interact(runall, Vam=widgets.FloatSlider(min=-90,max=-60,step=1,value=-72,description='Vam',layout=Layout(width='50%'), continuous_update = False),
                                  Vbm=widgets.FloatSlider(min=-30,max=0,step=1,value=-10,description='Vbm',layout=Layout(width='50%'), continuous_update = False),
                                  Vah=widgets.FloatSlider(min=-90,max=-60,step=1,value=-80,description='Vah',layout=Layout(width='50%'), continuous_update = False),
                                  Vbh=widgets.FloatSlider(min=-30,max=0,step=1,value=-20,description='Vbh',layout=Layout(width='50%'), continuous_update = False))


interactive(children=(FloatSlider(value=-72.0, continuous_update=False, description='Vam', layout=Layout(width…

#Question 1: See through the implementations of both the Python solver and the NEURON script above. Where and how are following parameters set in both scripts:<br>
1) Membrane capacitance?<br>
2) Initial membrane potential?<br>
3) End time of the simulation?<br>
<br><br>
#Question 2: See through the Python solver implementation above and the NEURON mechanism file (L4model.mod). Where and how are the following variables updated:<br>
1) Sodium current?<br>
2) Time derivative of gating variable $h$?<br>